In [ ]:
print("STARTING REVERSE PYTHIA MODEL FINE TUNING PROCESS...")

# PSC ENV settings

In [2]:
import os
CACHE_DIR = "/ocean/projects/cis250068p/jangabyl/caches" 
os.environ['HF_HOME'] = CACHE_DIR
os.environ['HF_DATASETS_CACHE']  = CACHE_DIR
os.environ['WANDB_DIR'] = CACHE_DIR


In [2]:
print("Huggingface directory setup complete...")
print(F"Saving cache at {CACHE_DIR}")

Huggingface directory setup complete...
Saving cache at /ocean/projects/cis250068p/jangabyl/caches


# Import packages

In [3]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import wandb
from config import *
from data import *
import wandb
wandb.init(project=f"{PROJECT_NAME}")

/ocean/projects/cis250068p/jangabyl/caches


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: janbol (11785_finetuning) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [21]:
print("Package import complete...")

Package import complete...


# Load the datasets (ZSOPT)

In [ ]:
print("Loading the dataset...")

In [4]:
dataset = load_summarization_datasets(
    subset_names=FLAN_SUBSET, 
    subset_frac=SUBSET_FRAC, 
    p=TASK_DIVERSITY_P)

  0%|          | 0/5 [00:00<?, ?it/s]

Processing niv2_zsopt_data...


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Dataset loaded successfully: niv2_zsopt_data
Original subset_niv2_zsopt_data size: 5030900
Found summarization examples in niv2_zsopt_data
Processing flan_zsopt_data...


...

Dataset loaded successfully: flan_zsopt_data
Original subset_flan_zsopt_data size: 38970972
Found summarization examples in flan_zsopt_data
Processing t0_zsopt_data...


Resolving data files:   0%|          | 0/131 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/131 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/44 [00:00<?, ?it/s]

Dataset loaded successfully: t0_zsopt_data
Original subset_t0_zsopt_data size: 41652381
Found summarization examples in t0_zsopt_data
Processing cot_zsopt_data...
Dataset loaded successfully: cot_zsopt_data
Original subset_cot_zsopt_data size: 95570
Found summarization examples in cot_zsopt_data
Processing dialog_zsopt_data...
Dataset loaded successfully: dialog_zsopt_data
Original subset_dialog_zsopt_data size: 2715160
Found summarization examples in dialog_zsopt_data
Combined dataset with 8930670 summarization examples


In [ ]:
'''from datasets import load_dataset
HF_KEY = HUGGING_FACE
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"  # Faster binary transfers
os.environ["HF_DATASETS_DOWNLOAD_MAX_WORKERS"] = "8"  # Increase worker count
os.environ["HF_HUB_NUM_WORKERS"] = "16"  # Concurrent HTTP connections
os.environ["HF_DATASETS_DOWNLOAD_MAX_WORKERS"] = "16"  # Concurrent file downloads
# List of subsets to download
subset_patterns = [
    "flan_zsopt_data/*.parquet",
    "niv2_zsopt_data/*.parquet",
    "t0_zsopt_data/*.parquet",
    "cot_zsopt_data/*.parquet",
    "dialog_zsopt_data/*.parquet"
]

# Load all subsets
dataset = load_dataset(
    "Open-Orca/FLAN",
    data_files=subset_patterns,
    use_auth_token="HF_KEY",
)

dataset = load_dataset(
    "Open-Orca/FLAN",
    data_files=subset_patterns)
# Access each subset (if split into separate datasets)
#print(dataset["flan_zsopt_data"][0])  # Example for FLAN subset'''

Resolving data files:   0%|          | 0/240 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/240 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/84 [00:00<?, ?it/s]

In [ ]:
print("Dataset loading complete...")

# Checking special tokens in Reverse model

In [ ]:
'''from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("afterless/reverse-pythia-160m")

# Check special tokens
print("EOS token:", tokenizer.eos_token)
print("EOS token ID:", tokenizer.eos_token_id)

print("BOS/SOS token:", tokenizer.bos_token)  
print("BOS/SOS token ID:", tokenizer.bos_token_id)
print("PAD token:", tokenizer.pad_token)
print("PAD token ID:", tokenizer.pad_token_id)
'''

EOS token: None
EOS token ID: None
BOS/SOS token: None
BOS/SOS token ID: None
PAD token: None
PAD token ID: None


In [ ]:
'''
tokenizer = AutoTokenizer.from_pretrained(
    "afterless/reverse-pythia-160m"
)
# Access the vocabulary dictionary
vocab = tokenizer.get_vocab()

# To get a list sorted by ID
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1])
for token, idx in sorted_vocab[:20]:  # show first 20
    print(f"Token ID: {idx} -> Token: {repr(token)}")
'''

Token ID: 0 -> Token: '<|endoftext|>'
Token ID: 1 -> Token: '<|padding|>'
Token ID: 2 -> Token: '!'
Token ID: 3 -> Token: '"'
Token ID: 4 -> Token: '#'
Token ID: 5 -> Token: '$'
Token ID: 6 -> Token: '%'
Token ID: 7 -> Token: '&'
Token ID: 8 -> Token: "'"
Token ID: 9 -> Token: '('
Token ID: 10 -> Token: ')'
Token ID: 11 -> Token: '*'
Token ID: 12 -> Token: '+'
Token ID: 13 -> Token: ','
Token ID: 14 -> Token: '-'
Token ID: 15 -> Token: '.'
Token ID: 16 -> Token: '/'
Token ID: 17 -> Token: '0'
Token ID: 18 -> Token: '1'
Token ID: 19 -> Token: '2'


# SCALING to LARGER FLAN

In [ ]:
print("Starting training...")

In [11]:
torch.cuda.empty_cache()  # Clears the cached memory
!nvidia-smi
import gc
gc.collect()


Tue Apr 22 12:31:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:15:00.0 Off |                    0 |
| N/A   28C    P0             55W /  300W |   26412MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

3904

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from datasets import load_dataset
import numpy as np
from datasets import Dataset  # Import Dataset class


# Configuration
model_name = "afterless/reverse-pythia-160m"

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizer setup
tokenizer.eos_token = '<|endoftext|>'
tokenizer.eos_token_id = 0
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token = tokenizer.eos_token

def preprocess_batch(examples):
    """Process batch with input/target flipping for reverse modeling"""
    # Format text
    texts = [f"{inp}\n{tgt} {tokenizer.eos_token}" 
             for inp, tgt in zip(examples['inputs'], examples['targets'])]
    
    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=1024,
        padding="max_length",
        #return_tensors="pt",

    )
    
    # Reverse sequences
    #tokenized['input_ids'] = torch.stack([torch.flip(ids, [0]) for ids in tokenized['input_ids']])
    #tokenized['attention_mask'] = torch.stack([torch.flip(mask, [0]) for mask in tokenized['attention_mask']])
    input_ids = tokenized["input_ids"]
    reversed_ids = [ids[::-1] for ids in input_ids]
    tokenized["input_ids"] = reversed_ids

    return tokenized

# Preprocess ALL data (non-streaming)
tokenized_dataset = dataset.map(
    preprocess_batch,
    batched=                        True,
    #batch_size=                    20000,
    num_proc=                       16
)

# Split dataset
train_val = tokenized_dataset.train_test_split(
    test_size=                      0.1,  # 10% for validation
    seed=                           42
)

# Training args
training_args = TrainingArguments(
    output_dir=                     OUTPUT_DIR,
    eval_strategy=                  "steps",
    eval_steps=                     EVAL_STEPS,
    per_device_train_batch_size=    BATCH_SIZE,
    per_device_eval_batch_size=     BATCH_SIZE,
    num_train_epochs=               NUM_EPOCHS,
    learning_rate=                  LEARNING_RATE,
    weight_decay=                   WEIGHT_DECAY,
    save_total_limit=               2,
    logging_dir=                    LOG_DIR,
    logging_steps=                  LOGGING_STEPS,
    fp16=                           True,
    gradient_accumulation_steps=    GRADIENT_ACCUMULATION_STEPS,
    report_to=                      "wandb"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_val["train"],
    eval_dataset=train_val["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Train
trainer.train()

# Save
print(f"Saving the fine tuned model at {OUTPUT_DIR}")
trainer.save_model("./pythia-finetuned-final")
tokenizer.save_pretrained("./pythia-finetuned-final")
print("Saving the fine tuned model is complete...")

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
print("Loading Fine Tuned model...")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "./pythia-finetuned-final"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
print("Generating sample output from Fine Tune model...")

In [92]:
def generate_flipped(prompt, max_length=200):
    # Tokenize without returning token_type_ids
    inputs = tokenizer(
        prompt, 
        return_tensors="pt",
        return_token_type_ids=False  # Key fix
    ).to(device)
    
    # Flip input sequence
    inputs['input_ids'] = t.flip(inputs.input_ids, (1,))
    
    with t.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.4,
            top_k=50,
            top_p=0.92,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    
    # Flip back and decode
    return tokenizer.decode(t.flip(output, (1,))[0], skip_special_tokens=True)
print(generate_flipped("Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 says.  They now own four children, and he believes they are just like their parents.  "We love our children because we did not even see them growing up," he says.   He also plans to take a job at a Â£1million annual salary, from which he estimates he will earn around $200,000 over the next six months.  His family were shocked when he spoke publicly for the first time, saying  "You must prove yourself or you'll never make any money". But he later said: "I couldn't do it - but I was really worried about me."  He had signed a new three-year contract earlier this year and is yet to decide on where he would spend the rest of his life. He may have been contacted by an agent who has agreed to stay with him for five years, in a deal that could be worth more than $100 million.
Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday
